# Language Model Loading

In [ ]:
import sys
sys.path.append('../french-crs')

from text2corefchains import udpipe_spacy_lang_model, mentions2chains
model=udpipe_spacy_lang_model("../external language models/french-ud-2.0-170801.udpipe",
                              "fr_core_news_md")

# Text Parsing

In [ ]:
model.tokenize("à mettre en œuvre de le protectionnisme intelligent à mettre en avant de le patriotisme économique pour donner un avantage à les entreprises françaises dans la commande publique voilà tout cela . Le patriotisme économique qui n ’ a jamais été mis en œuvre le protectionnisme intelligent la défiscalisation de les heures supplémentaires la suppression de le travail détaché la baisse de les charges mais exclusivement pour les TPE PME . Il met en place un patriotisme économique un protectionnisme intelligent il dit à les constructeurs américains si vous voulez aller faire vos voitures à l ’ étranger construire une voiture à l’ étranger alors vous paierez une taxe en les réimportant à les Etats-Unis . D ’ autant que évidemment ce que fait Trump m ’ intéresse et pour cause puisqu ’ il met en place la politique que j’ appelle de mes vœux depuis très longtemps et notamment la politique de patriotisme économique de protectionnisme intelligent")    
model.parse_sentences_json()
model.update_json_sents_with_spacy_entities(operation="in")
model.update_json_sents_with_mentions(operation="==")

# Pairs & Features Building

In [ ]:
chains_generator=mentions2chains(model.mentions_ids_in_json_sents)
chains_generator.generate_mention_pairs(window_size=30, iteration=3)
chains_generator.generate_json_mention_pairs(model.mentions_ids_in_json_sents,model.sentences_json)
chains_generator.json_mention_pairs2dataframe(save_file=True, file_path="./unseen_data.xlsx")

# Coreference Prediction

In [ ]:
chains_generator.json_mention_pairs2dataframe(save_file=True, file_path="./unseen_data.xlsx")
chains_generator.init_data_model(model_name="../pre-trained language models/Model_ANCOR_Representative_INDIRECTE.model", 
                                 input_method=".xlsx",
                                 file_path="./unseen_data.xlsx",
                                 column_outcome="Prediction",
                                 threshold=0.5)

chains_generator.apply_model_to_dataset()

# Coreference Chains Building

In [ ]:
from pairs2chains import chains_builder

chains = chains_builder("./unseen_data.xlsx",
                        "./unseen_data.xlsx",
                        "Prediction",
                        "Prediction",
                        0.5)

chains.generate_gold_model_json_output(mode="test")
chains.coref_chains_pred_json

# Coreference Chains Visualizer

In [ ]:
chains.context_visualizer(model.sentences_json,
                          context_left_window=3,
                          context_right_window=3,
                          theme_left_window=5,
                          theme_right_window=4,
                          file_path="context_visualized.xlsx")


# Experiment I : Running through 12 models

In [ ]:
import glob
from pairs2chains import chains_builder
threshold=0.5


PATH="./Optimized_Models/"


# Uncomment for automatic files_list instead of manually creation of list

# files_list=[file[file.rfind("/")+1:] for file in glob.glob(PATH+"*.model")]


files_list=[
#             'Model_ANCOR_Ballanced_Full.model',
#             'Model_ANCOR_Ballanced_ANAPHORE.model',
#             'Model_ANCOR_Ballanced_DIRECTE.model',
            'Model_ANCOR_Ballanced_INDIRECTE.model',
#             'Model_ANCOR_Representative_Full.model',
#             'Model_ANCOR_Representative_ANAPHORE.model',
#             'Model_ANCOR_Representative_DIRECTE.model',
            'Model_ANCOR_Representative_INDIRECTE.model'
#             'Model_ANCOR_Window_Full.model',
#             'Model_ANCOR_Window_ANAPHORE.model',
#             'Model_ANCOR_Window_DIRECTE.model',
#             'Model_ANCOR_Window_INDIRECTE.model'
]


for file in files_list:
    
    chains_generator=mentions2chains(model.mentions_ids_in_json_sents)
    chains_generator.generate_mention_pairs(window_size=30, iteration=3)
    chains_generator.generate_json_mention_pairs(model.mentions_ids_in_json_sents,model.sentences_json)
    chains_generator.json_mention_pairs2dataframe(save_file=True, file_path="./unseen_data.xlsx")
    
    chains_generator.init_data_model(model_name="../pre-trained language models/"+file, 
                                 input_method=".xlsx",
                                 file_path="./unseen_data.xlsx",
                                 column_outcome="Prediction",
                                 threshold=threshold)

    chains_generator.apply_model_to_dataset()
    
    from pairs2chains import chains_builder

    chains = chains_builder("./unseen_data.xlsx",
                            "./unseen_data.xlsx",
                            "Prediction",
                            "Prediction",
                            0.5)

    chains.generate_gold_model_json_output(mode="test")
    
    print("---------------------------------------------------------------")
    print("Model: "+file)
    for chain in chains.coref_chains_pred_json:
        print(chain)
        print(chains.coref_chains_pred_json[chain])
    print("---------------------------------------------------------------")
    

In [ ]:
files_list

# Experiment II : Running through different thresholds

In [ ]:
from pairs2chains import chains_builder
threshold=1.05

for iterate in range(0,20):
    threshold-=0.05
    chains = chains_builder("./unseen_data.xlsx",
                            "./unseen_data.xlsx",
                            "Prediction",
                            "Prediction",
                            threshold)
    coreference_chain_built=chains.generate_coreference_chains(chains.gold_dataframe, 
                                                               chains.gold_column, 
                                                               mode="test")
    
    print("---------------------------------------------------------------")
    print("Threshold :"+str(round(threshold,2)))
    for chain in coreference_chain_built:
        print(chain)
        print(coreference_chain_built[chain])
    print("---------------------------------------------------------------")